In [1]:
import pandas as pd
import chess
import chess.pgn
import chess.engine
import asyncio

In [2]:
eng = chess.engine.SimpleEngine.popen_uci("../../stockfish/stockfish")

In [3]:
p = open("../data/RuyLopezMarshall.pgn")

In [4]:
games_id = []
games = []

while True:
    offset = p.tell()
    headers = chess.pgn.read_headers(p)
    if headers is None:
        break
    games_id.append(offset)

for offset in games_id:
    p.seek(offset)
    games.append(chess.pgn.read_game(p))

In [5]:
dados = []
count = -1

for game in games[:2]:
    count += 1
    row = {}
    head = game.headers

    if head['Site'].endswith(' INT') or head['Site'] == '?':
        continue

    row['id'] = games_id[count]
    row['Event'] = head['Event']
    row['Site'] = head['Site']
    row['Date'] = head['Date'].split('.')[0]
    row['White'] = head['White']
    row['Black'] = head['Black']
    row['Result'] = head['Result']
    row['BlackElo'] = head['BlackElo']
    row['ECO'] = 'C89'
    row['WhiteElo'] = head['WhiteElo']
    row['White Short Castle'] = 0
    row['White Long Castle'] = 0
    row['Black Short Castle'] = 0
    row['Black Long Castle'] = 0
    row['Opposite side castling'] = 0
    row['Evals'] = []

    # quem ganhou
    if head['Result'] == '0-1':
        row['WhiteWon'] = 0
        row['BlackWon'] = 1
    elif head['Result'] == '1-0':
        row['WhiteWon'] = 1
        row['BlackWon'] = 0
    else:
        row['WhiteWon'] = 0
        row['BlackWon'] = 0

    # qntd de moves
    final_position = game[0].end().board()
    row['Moves'] = final_position.fullmove_number

    # qntd de peças restantes
    final_fen = final_position.board_fen()
    white_count = 0
    black_count = 0
    white_pieces = ['P', 'N', 'B', 'R', 'Q']
    black_pieces = ['p', 'n', 'b', 'r', 'q']
    white_has_queen = 0
    black_has_queen = 0

    for char in range(0, len(final_fen)):
        if final_fen[char] in white_pieces:
            white_count += 1
        elif final_fen[char] in black_pieces:
            black_count += 1

        if final_fen[char] == 'Q':
            white_has_queen = 1
        elif final_fen[char] == 'q':
            black_has_queen = 1

    row['WhiteFinalPiecesCount'] = white_count
    row['BlackFinalPiecesCount'] = black_count
    row['FinalPiecesCount'] = white_count + black_count

    if white_has_queen == 0 and black_has_queen == 0:
        row['QueenTrade'] = 1
    else:
        row['QueenTrade'] = 0

    board = game[0].board()
    info = eng.analyse(board, chess.engine.Limit(time=0.1))
    row['Evals'].append(info['score'].white().score(mate_score=100000))

    for move in game[0].mainline_moves():        
        board.push(move)
        info = eng.analyse(board, chess.engine.Limit(time=0.1))
        row['Evals'].append(info['score'].white().score(mate_score=100000))

        m = move.uci()

        flag_wking_move = 0
        flag_bking_move = 0

        if m[:2] == 'e1' and flag_wking_move == 0:
            flag_wking_move = 1

            if m == 'e1g1':
                row['White Short Castle'] = 1
            elif m == 'e1c1':
                row['White Long Castle'] = 1

        elif m[:2] == 'e8' and flag_bking_move == 0:
            flag_bking_move = 1

            if m == 'e8g8':
                row['Black Short Castle'] = 1
            elif m == 'e8c8':
                row['Black Long Castle'] = 1

        if (row['White Short Castle'] == 1 and row['Black Long Castle'] == 1) or \
                (row['Black Short Castle'] == 1 and row['White Long Castle'] == 1):
            row['Opposite side castling'] = 1
    
    dados.append(row)

eng.quit()
df = pd.DataFrame(dados)


In [6]:
print(df['Evals'][0])

[35, 29, 34, 41, 35, 39, 46, 50, 29, 67, 60, 84, 64, 57, 36, 46, 63, 55, 0, -4, -1, 53, 22, 55, -61, -26, -45, -33, -118, -83, -145, -58, -99, -228, -339, -51, -43, -37, -42, -20, -21, 0, 8, -2, -50, -28, -60, -60, -53, -28, -89, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -7, -23, -37, -55, -50, -58, -27, -134, -135, -319, -254, -284, -308, -322, -326, -364, -334, -357, -374, -333, -399, -383, -391, -387, -404, -402, 0, -382, -219, -227, -189, -229, -274, -268, -286, -295, -297, -299, -167, -175, -77, -199, -209, -207, -218, -214, -225, -241, -189, -227, -92, -84, -112, -108, -120, -147, -107, -248, -204, -276, -201, -332, -142, -182, -173, -171, -112, -181, -215, -275, -318, -326, -383, -398, -494, -536, -537, -641, -589, -648, -657, -745, -730, -806, -833, -853, -874, -886, -915, -1025, -1028, -1147, -1169, -1167, -1223, -1251, -1259, -1675, -1857, -1969, -2938, -6173, -99977, -99987, -99988]


In [7]:
# corrigir places/Sites
def string_analyzer(row):
    s = row['Site']
    splitted_s = s.split(' ')
    first_part = splitted_s[:-1]
    last_part = splitted_s[-1]
    city = ''

    if last_part.isupper() and len(splitted_s) > 1:
        for w in first_part:
            city += w + ' '
    else:
        city = s

    row['Site'] = city

    return row


df = df.apply(lambda row: string_analyzer(row=row), axis=1)
